In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz

In [ ]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

In [9]:
!gzip -d green_tripdata_2019-09.csv.gz

In [10]:
!ls

green_tripdata_2019-09.csv  HW1.ipynb  taxi+_zone_lookup.csv


In [11]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("main")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.shuffle.service.enabled", "true")\
    .getOrCreate()

In [17]:
green_taxi = spark.read.csv("green_tripdata_2019-09.csv", header="true", inferSchema="true")

In [20]:
print(green_taxi.printSchema())

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

None


In [21]:
from pyspark.sql.types import *

In [29]:
green_taxi =  green_taxi.withColumn("lpep_pickup_datetime",green_taxi["lpep_pickup_datetime"].cast(TimestampType())) \
                .withColumn("lpep_dropoff_datetime",green_taxi["lpep_dropoff_datetime"].cast(TimestampType()))                

In [68]:
trip_largest = green_taxi.orderBy("trip_distance", ascending = False).first()

In [69]:
print(trip_largest)

Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2019, 9, 26, 19, 32, 52), lpep_dropoff_datetime=datetime.datetime(2019, 9, 27, 1, 50, 27), store_and_fwd_flag='N', RatecodeID=4, PULocationID=265, DOLocationID=16, passenger_count=1, trip_distance=341.64, fare_amount=1761.0, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=1762.8, payment_type=2, trip_type=1, congestion_surcharge=0.0)


In [63]:
df_date = green_taxi.where(datediff(green_taxi["lpep_pickup_datetime"], lit("2019-09-18 00:00:00")) == 0).where(datediff(green_taxi["lpep_dropoff_datetime"], lit("2019-09-18 00:00:00"))== 0).count()

In [64]:
print(df_date)

15612
